In [329]:
import pandas as pd
from scipy import stats
import json
import copy


pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [330]:

class JSONFlattener:
    def __init__(self, depth = 1, prefix = ''):
        self.prefix = prefix
        self.depth = depth
    
    def fit(x):
        self.possible_columns = list()
        for i in x:
            self.possible_columns.extend(list(i.keys()))
        self.possible_columns = sorted(list(set(self.possible_columns)))
        self.col_names = dict()
        for i in self.possible_columns:
            if self.prefix:
                self.col_names[i] = '{0}_{1}'.format(prefix, i)
            else:
                self.col_names[i] = i
        
    def transform(x):
        data_dicts = list()
        for i in x:
            temp_dict = dict()
            for j in self.possible_columns:
                temp_dict[self.col_names[j]] = i.get(j)
        return pd.DataFrame(data_dicts)


In [331]:
with open(r'C:\Users\TristanDelforge\Documents\data_files/7.json', 'r') as f:
    past_results = json.load(f)
past_results.keys()

dict_keys(['models', 'datasets', 'results'])

In [333]:
model_df = pd.DataFrame(past_results['models'])
model_df.head(20)

model_dicts = list()
for i in past_results['models']:
    new_dict = {'model_id':i['model_id'],
               'model_type':i['model_type'],
               }
    new_dict.update(i['model_params'])
        
    model_dicts.append(new_dict)
model_df = pd.DataFrame(model_dicts)
model_df.shape

(140, 9)

In [334]:

transformation_dicts = list()
general_state_dicts = list()

for i in past_results['datasets']:
    for j in i:
        new_transformation_dict = {'dataset_id': j['dataset_id'],
                                   'transformation_id': j['transformation_id'],
                                  'transformation_type': j['transformation_type']}
        new_transformation_dict.update(j['transformation_parameters'])
        for n, k in enumerate(j['input_column_descriptions']):
            d_copy = copy.deepcopy(k)
            for c in k.keys():
                d_copy['input_column_{0}_{1}'.format(n, c)] = d_copy[c]
                del d_copy[c]
            new_transformation_dict.update(d_copy)
            
        transformation_dicts.append(new_transformation_dict)
        
        new_general_state_dict = {'dataset_id': j['dataset_id'],
                                   'transformation_id': j['transformation_id']}
        new_general_state_dict.update(j['general_description'])
        general_state_dicts.append(new_general_state_dict)
        
transformations_df = pd.DataFrame(transformation_dicts)
general_state_df = pd.DataFrame(general_state_dicts)
transformations_df.shape, general_state_df.shape, model_df.shape


((1303, 35), (1303, 65), (140, 9))

In [ ]:
result_df[(result_df['dataset_id'] == '260f8a9a8994498982d42b7eed231a7b')&(result_df['model_id'] == '21c63271de524214967f40336aed68d5')]

In [359]:
 [i for i in past_results['results'] if i['dataset_id'] == '260f8a9a8994498982d42b7eed231a7b' and i['model_id'] == '21c63271de524214967f40336aed68d5']

[{'model_id': '21c63271de524214967f40336aed68d5',
  'dataset_id': '260f8a9a8994498982d42b7eed231a7b',
  'problem_type': 'regression',
  'strategy': 'run_random_pipelines',
  'validation_metrics': {'mean_absolute_error': 38163.34505392148,
   'mean_squared_error': 4529849436.304764,
   'r2_score': 0.009006941572918703,
   'mean_squared_log_error': 3.60097115117623},
  'test_metrics': {'mean_absolute_error': 38163.34505392148,
   'mean_squared_error': 4529849436.304764,
   'r2_score': 0.009006941572918703,
   'mean_squared_log_error': 3.60097115117623},
  'success': 1},
 {'model_id': '21c63271de524214967f40336aed68d5',
  'dataset_id': '260f8a9a8994498982d42b7eed231a7b',
  'problem_type': 'regression',
  'strategy': 'run_random_pipelines',
  'validation_metrics': {'mean_absolute_error': 38199.9561457893,
   'mean_squared_error': 4534247860.430823,
   'r2_score': 0.008044700368606605,
   'mean_squared_log_error': 3.618280450982836},
  'test_metrics': {'mean_absolute_error': 38199.956145789

In [356]:
result_dicts = list()
for i in past_results['results']:
    new_dict = {'model_id':i['model_id'],
               'dataset_id':i['dataset_id'],
               'validation_metrics':i['validation_metrics']['mean_absolute_error']}
        
    result_dicts.append(new_dict)
result_df = pd.DataFrame(result_dicts)
result_df.shape

(1400, 3)

In [357]:
result_df = result_df.dropna(subset = ['validation_metrics'])
result_df.sort_values(['model_id', 'dataset_id']).head(20)

,model_id,dataset_id,validation_metrics
1201,00b5287b2bd34836965749d5e58e6bba,d791a57fe5e34f08b92b4e4878fcd8b3,3.785205e+04
1211,00b5287b2bd34836965749d5e58e6bba,d791a57fe5e34f08b92b4e4878fcd8b3,3.717844e+04
1221,00b5287b2bd34836965749d5e58e6bba,d791a57fe5e34f08b92b4e4878fcd8b3,3.552915e+04
1231,00b5287b2bd34836965749d5e58e6bba,d791a57fe5e34f08b92b4e4878fcd8b3,3.822547e+04
1241,00b5287b2bd34836965749d5e58e6bba,d791a57fe5e34f08b92b4e4878fcd8b3,3.847939e+04
1251,00b5287b2bd34836965749d5e58e6bba,d791a57fe5e34f08b92b4e4878fcd8b3,3.846644e+04
1261,00b5287b2bd34836965749d5e58e6bba,d791a57fe5e34f08b92b4e4878fcd8b3,3.820363e+04
1271,00b5287b2bd34836965749d5e58e6bba,d791a57fe5e34f08b92b4e4878fcd8b3,3.699154e+04
1281,00b5287b2bd34836965749d5e58e6bba,d791a57fe5e34f08b92b4e4878fcd8b3,3.805539e+04
1291,00b5287b2bd34836965749d5e58e6bba,d791a57fe5e34f08b92b4e4878fcd8b3,3.763890e+04


In [338]:
set(general_state_df.columns)&set(transformations_df.columns)

{'dataset_id', 'transformation_id'}

In [339]:
general_state_df.head()

,dataset_id,transformation_id,rows,columns,feature_columns_size,target_mean,target_var,target_skew,target_perc_unique,target_kurtosis,mean_mean,mean_var,mean_skew,mean_perc_unique,mean_kurtosis,var_mean,var_var,var_skew,var_perc_unique,var_kurtosis,skew_mean,skew_var,skew_skew,skew_perc_unique,skew_kurtosis,perc_unique_mean,perc_unique_var,perc_unique_skew,perc_unique_perc_unique,perc_unique_kurtosis,kurtosis_mean,kurtosis_var,kurtosis_skew,kurtosis_perc_unique,kurtosis_kurtosis,perc_of_values_mode_mean,perc_of_values_mode_var,perc_of_values_mode_skew,perc_of_values_mode_perc_unique,perc_of_values_mode_kurtosis,target_slope_mean,target_slope_var,target_slope_skew,target_slope_perc_unique,target_slope_kurtosis,target_r_value_mean,target_r_value_var,target_r_value_skew,target_r_value_perc_unique,target_r_value_kurtosis,target_p_value_mean,target_p_value_var,target_p_value_skew,target_p_value_perc_unique,target_p_value_kurtosis,nunique_mean,nunique_var,nunique_skew,nunique_perc_unique,nunique_kurtosis,target_f_stat_mean,target_f_stat_var,target_f_stat_skew,target_f_stat_perc_unique,target_f_stat_kurtosis
0,260f8a9a8994498982d42b7eed231a7b,1c7a2a6e7ae14c06b025ffb1659d73e4,13332,124,124,0.008065,11.135529,11.135529,0.008065,119.008130,1792.789004,6.402077,6.402077,0.322581,36.016924,-2.690683,2.491526,2.491526,0.322581,8.988515,-2.690683,2.491526,2.491526,0.322581,8.988515,0.004468,6.402689,6.402689,0.032258,36.021642,14.754817,5.331932,5.331932,0.322581,27.988896,0.366900,0.729624,0.729624,0.895161,-0.156190,2.936962,-0.735893,-0.735893,0.322581,-1.180441,0.023350,5.590404,5.590404,0.322581,30.057257,0.037184,5.239341,5.239341,0.951613,25.135290,411.53012,3.831147,3.831147,0.250000,14.60629,38.765662,6.121918,6.121918,0.661290,42.356203
1,260f8a9a8994498982d42b7eed231a7b,baa100d3074540338e44efaff8085d94,13332,125,125,0.008000,11.180340,11.180340,0.008000,120.008065,1738.697457,6.479498,6.479498,0.328000,37.014670,-2.648222,2.435777,2.435777,0.328000,8.866827,-2.648222,2.435777,2.435777,0.328000,8.866827,0.004397,6.479998,6.479998,0.040000,37.018569,14.398459,5.379448,5.379448,0.328000,28.630461,0.364677,0.731565,0.731565,0.896000,-0.157388,2.785495,-0.686100,-0.686100,0.328000,-1.222447,0.022044,5.654107,5.654107,0.328000,30.855814,0.036889,5.262174,5.262174,0.952000,25.377291,411.53012,3.831147,3.831147,0.248000,14.60629,38.765662,6.121918,6.121918,0.656000,42.356203
2,260f8a9a8994498982d42b7eed231a7b,80fa7737dd3142919695ede8a8fcb8be,13332,126,126,0.007937,11.224972,11.224972,0.007937,121.008000,1698.298729,6.556145,6.556145,0.333333,38.013552,-2.567062,2.276786,2.276786,0.333333,7.994429,-2.567062,2.276786,2.276786,0.333333,7.994429,0.004305,6.556669,6.556669,0.047619,38.017691,14.056377,5.426552,5.426552,0.333333,29.270862,0.364807,0.732505,0.732505,0.888889,-0.136692,-24.286229,-6.532504,-6.532504,0.333333,37.825256,0.020873,5.718449,5.718449,0.333333,31.665242,0.036605,5.284920,5.284920,0.952381,25.619386,411.53012,3.831147,3.831147,0.246032,14.60629,38.765662,6.121918,6.121918,0.650794,42.356203
3,260f8a9a8994498982d42b7eed231a7b,d340de3a6f8f4d91a0f91ce2241be1b1,13332,162,162,0.006173,12.727922,12.727922,0.006173,157.006211,924.394273,8.885577,8.885577,0.277778,73.983410,-0.899716,0.583056,0.583056,0.277778,1.002514,-0.899716,0.583056,0.583056,0.277778,1.002514,0.002355,8.882765,8.882765,0.049383,73.951670,8.195573,6.615734,6.615734,0.277778,48.827930,0.500104,0.425449,0.425449,0.709877,-1.250925,94.624441,-5.844324,-5.844324,0.271605,36.738532,0.011284,7.768826,7.768826,0.277778,62.007701,0.230379,1.259524,1.259524,0.759259,-0.378646,411.53012,3.831147,3.831147,0.191358,14.60629,38.765662,6.121918,6.121918,0.506173,42.356203
4,260f8a9a8994498982d42b7eed231a7b,6c8b45cefea34f6b9496b182a0ec34c2,13332,175,175,0.005714,13.228757,13.228757,0.005714,170.005747,793.779121,9.588688,9.588688,0.268571,86.974656,-0.431375,0.202865,0.202865,0.268571,0.262808,-0.431375,0.202865,0.202865,0.268571,0.262808,0.002043,9.585523,9.585523,0.0514

In [340]:
model_df.head()

,model_id,model_type,alpha,l1_ratio,objective,boosting_type,num_leaves,learning_rate,n_estimators
0,21c63271de524214967f40336aed68d5,ElasticNet,1.607837,0.600348,NaN,NaN,NaN,NaN,NaN
1,227ad42c8e84480db5ce1fce8ca2f846,ElasticNet,1.826048,0.193469,NaN,NaN,NaN,NaN,NaN
2,1472d7ca77ce412ab8d443c95e570923,ElasticNet,1.956714,0.048165,NaN,NaN,NaN,NaN,NaN
3,95d166e820104ae6ad70522ce807bb2f,LGBMRegressor,NaN,NaN,mape,goss,40.0,0.160605,87.0
4,b4633c3faef44fb985abf8b496a67f21,ElasticNet,1.802553,0.554751,NaN,NaN,NaN,NaN,NaN


In [322]:
transformations_df.head()

,dataset_id,transformation_id,transformation_type,input_column_0_target,input_column_0_type,input_column_0_mean,input_column_0_var,input_column_0_skew,input_column_0_perc_unique,input_column_0_kurtosis,input_column_0_perc_of_values_mode,input_column_0_target_slope,input_column_0_target_r_value,input_column_0_target_p_value,input_column_1_target,input_column_1_type,input_column_1_mean,input_column_1_var,input_column_1_skew,input_column_1_perc_unique,input_column_1_kurtosis,input_column_1_perc_of_values_mode,input_column_1_target_slope,input_column_1_target_r_value,input_column_1_target_p_value,num_of_output_columns,input_column_0_nunique,input_column_0_target_f_stat,max_features,norm,analyzer,max_df,binary,use_idf,ngram_range
0,74de0f823949466494fd7fb33a51e40e,332ae54a8db4479485731f5aef257981,standard_scaler,0,numeric,-147.807381,-1.946423,-1.946423,0.000525,1.787886,0.151290,5.227856,0.027757,1.349673e-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,74de0f823949466494fd7fb33a51e40e,c16f0b42e5b14131ad81bc2612c66267,identity,0,numeric,-39.120762,-4.326942,-4.326942,0.000825,16.721221,0.136064,11.760928,0.036684,2.268453e-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,74de0f823949466494fd7fb33a51e40e,11c3566c5f1c4a99b4c484f1806dcc92,sum,0,numeric,-30.189769,-4.986001,-4.986001,0.000750,22.860222,0.120987,7.712221,0.021314,1.385163e-02,0.0,numeric,-314.435494,-0.784512,-0.784512,0.000375,-1.384663,0.317432,-11.950664,-0.082398,1.582438e-21,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,74de0f823949466494fd7fb33a51e40e,79a90d466ab4449a8654a34ae201e171,one_hot_encoding,0,string,NaN,NaN,NaN,NaN,NaN,0.388989,NaN,NaN,8.055766e-22,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,37.0,7.0,18.729886,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,74de0f823949466494fd7fb33a51e40e,c645b761280e4b47a774d6cfb428f4b9,max,0,numeric,-314.435494,-0.784512,-0.784512,0.000375,-1.384663,0.317432,-11.950664,-0.082398,1.582438e-21,0.0,numeric,-329.918167,-0.704300,-0.704300,0.000525,-1.504039,0.333933,-8.896932,-0.062250,6.296600e-13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [323]:
result_df.head()

,model_id,dataset_id,validation_metrics
0,62101a63d4f44306ab0b74f2fdf9d33e,74de0f823949466494fd7fb33a51e40e,38089.386057
1,a1a58eab86ca4b88b4a8923792a368a6,74de0f823949466494fd7fb33a51e40e,38050.691729
2,51d11448d9bc4d5e8ad64cc3606c7740,74de0f823949466494fd7fb33a51e40e,59766.915511
3,d6a72b76c2ad44d9b54d522cbcd42a7b,74de0f823949466494fd7fb33a51e40e,36228.136096
4,87ceb07e4e1142a9b43cf3a9db21d567,74de0f823949466494fd7fb33a51e40e,38478.969693


In [347]:
merge_df = general_state_df.merge(transformations_df)
merge_df.head()

,dataset_id,transformation_id,rows,columns,feature_columns_size,target_mean,target_var,target_skew,target_perc_unique,target_kurtosis,mean_mean,mean_var,mean_skew,mean_perc_unique,mean_kurtosis,var_mean,var_var,var_skew,var_perc_unique,var_kurtosis,skew_mean,skew_var,skew_skew,skew_perc_unique,skew_kurtosis,perc_unique_mean,perc_unique_var,perc_unique_skew,perc_unique_perc_unique,perc_unique_kurtosis,kurtosis_mean,kurtosis_var,kurtosis_skew,kurtosis_perc_unique,kurtosis_kurtosis,perc_of_values_mode_mean,perc_of_values_mode_var,perc_of_values_mode_skew,perc_of_values_mode_perc_unique,perc_of_values_mode_kurtosis,target_slope_mean,target_slope_var,target_slope_skew,target_slope_perc_unique,target_slope_kurtosis,target_r_value_mean,target_r_value_var,target_r_value_skew,target_r_value_perc_unique,target_r_value_kurtosis,target_p_value_mean,target_p_value_var,target_p_value_skew,target_p_value_perc_unique,target_p_value_kurtosis,nunique_mean,nunique_var,nunique_skew,nunique_perc_unique,nunique_kurtosis,target_f_stat_mean,target_f_stat_var,target_f_stat_skew,target_f_stat_perc_unique,target_f_stat_kurtosis,transformation_type,input_column_0_target,input_column_0_type,input_column_0_mean,input_column_0_var,input_column_0_skew,input_column_0_perc_unique,input_column_0_kurtosis,input_column_0_perc_of_values_mode,input_column_0_target_slope,input_column_0_target_r_value,input_column_0_target_p_value,input_column_1_target,input_column_1_type,input_column_1_mean,input_column_1_var,input_column_1_skew,input_column_1_perc_unique,input_column_1_kurtosis,input_column_1_perc_of_values_mode,input_column_1_target_slope,input_column_1_target_r_value,input_column_1_target_p_value,input_column_0_nunique,input_column_0_target_f_stat,max_features,norm,analyzer,max_df,binary,use_idf,ngram_range,num_of_output_columns
0,260f8a9a8994498982d42b7eed231a7b,1c7a2a6e7ae14c06b025ffb1659d73e4,13332,124,124,0.008065,11.135529,11.135529,0.008065,119.008130,1792.789004,6.402077,6.402077,0.322581,36.016924,-2.690683,2.491526,2.491526,0.322581,8.988515,-2.690683,2.491526,2.491526,0.322581,8.988515,0.004468,6.402689,6.402689,0.032258,36.021642,14.754817,5.331932,5.331932,0.322581,27.988896,0.366900,0.729624,0.729624,0.895161,-0.156190,2.936962,-0.735893,-0.735893,0.322581,-1.180441,0.023350,5.590404,5.590404,0.322581,30.057257,0.037184,5.239341,5.239341,0.951613,25.135290,411.53012,3.831147,3.831147,0.250000,14.60629,38.765662,6.121918,6.121918,0.661290,42.356203,sum,0,numeric,-331.248575,-0.704291,-0.704291,0.000525,-1.504039,0.333933,-8.964766,-0.062599,4.675794e-13,0.0,numeric,-147.807381,-1.946423,-1.946423,0.000525,1.787886,0.15129,5.227856,0.027757,0.00135,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,260f8a9a8994498982d42b7eed231a7b,baa100d3074540338e44efaff8085d94,13332,125,125,0.008000,11.180340,11.180340,0.008000,120.008065,1738.697457,6.479498,6.479498,0.328000,37.014670,-2.648222,2.435777,2.435777,0.328000,8.866827,-2.648222,2.435777,2.435777,0.328000,8.866827,0.004397,6.479998,6.479998,0.040000,37.018569,14.398459,5.379448,5.379448,0.328000,28.630461,0.364677,0.731565,0.731565,0.896000,-0.157388,2.785495,-0.686100,-0.686100,0.328000,-1.222447,0.022044,5.654107,5.654107,0.328000,30.855814,0.036889,5.262174,5.262174,0.952000,25.377291,411.53012,3.831147,3.831147,0.248000,14.60629,38.765662,6.121918,6.121918,0.656000,42.356203,dictionary_encode,0,string,NaN,NaN,NaN,NaN,NaN,0.381113,NaN,NaN,6.862342e-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.0,2.961435,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,260f8a9a8994498982d42b7eed231a7b,80fa7737dd3142919695ede8a8fcb8be,13332,126,126,0.007937,11.224972,11.224972,0.007937,121.008000,1698.298729,6.556145,6.556145,0.333333,38.013552,-2.567062,2.276786,2.276786,0.333333,7.994429,-2.567062,2.276786,2.276786,0.333333,7.994429,0.004305,6.556669,6.556669,0.047619,38.017691,14.056377,5.426552,5.426552,0.333333,29.270862,0.364807,0.732505,0.732505,0.888889,-0.136692,-24.286229,-6.532504,-6.532504,0.333333,37.825256,0.020873,5.718449,

In [348]:
merge_df = merge_df.merge(result_df)
merge_df.head()

,dataset_id,transformation_id,rows,columns,feature_columns_size,target_mean,target_var,target_skew,target_perc_unique,target_kurtosis,mean_mean,mean_var,mean_skew,mean_perc_unique,mean_kurtosis,var_mean,var_var,var_skew,var_perc_unique,var_kurtosis,skew_mean,skew_var,skew_skew,skew_perc_unique,skew_kurtosis,perc_unique_mean,perc_unique_var,perc_unique_skew,perc_unique_perc_unique,perc_unique_kurtosis,kurtosis_mean,kurtosis_var,kurtosis_skew,kurtosis_perc_unique,kurtosis_kurtosis,perc_of_values_mode_mean,perc_of_values_mode_var,perc_of_values_mode_skew,perc_of_values_mode_perc_unique,perc_of_values_mode_kurtosis,target_slope_mean,target_slope_var,target_slope_skew,target_slope_perc_unique,target_slope_kurtosis,target_r_value_mean,target_r_value_var,target_r_value_skew,target_r_value_perc_unique,target_r_value_kurtosis,target_p_value_mean,target_p_value_var,target_p_value_skew,target_p_value_perc_unique,target_p_value_kurtosis,nunique_mean,nunique_var,nunique_skew,nunique_perc_unique,nunique_kurtosis,target_f_stat_mean,target_f_stat_var,target_f_stat_skew,target_f_stat_perc_unique,target_f_stat_kurtosis,transformation_type,input_column_0_target,input_column_0_type,input_column_0_mean,input_column_0_var,input_column_0_skew,input_column_0_perc_unique,input_column_0_kurtosis,input_column_0_perc_of_values_mode,input_column_0_target_slope,input_column_0_target_r_value,input_column_0_target_p_value,input_column_1_target,input_column_1_type,input_column_1_mean,input_column_1_var,input_column_1_skew,input_column_1_perc_unique,input_column_1_kurtosis,input_column_1_perc_of_values_mode,input_column_1_target_slope,input_column_1_target_r_value,input_column_1_target_p_value,input_column_0_nunique,input_column_0_target_f_stat,max_features,norm,analyzer,max_df,binary,use_idf,ngram_range,num_of_output_columns,model_id,validation_metrics
0,260f8a9a8994498982d42b7eed231a7b,1c7a2a6e7ae14c06b025ffb1659d73e4,13332,124,124,0.008065,11.135529,11.135529,0.008065,119.00813,1792.789004,6.402077,6.402077,0.322581,36.016924,-2.690683,2.491526,2.491526,0.322581,8.988515,-2.690683,2.491526,2.491526,0.322581,8.988515,0.004468,6.402689,6.402689,0.032258,36.021642,14.754817,5.331932,5.331932,0.322581,27.988896,0.3669,0.729624,0.729624,0.895161,-0.15619,2.936962,-0.735893,-0.735893,0.322581,-1.180441,0.02335,5.590404,5.590404,0.322581,30.057257,0.037184,5.239341,5.239341,0.951613,25.13529,411.53012,3.831147,3.831147,0.25,14.60629,38.765662,6.121918,6.121918,0.66129,42.356203,sum,0,numeric,-331.248575,-0.704291,-0.704291,0.000525,-1.504039,0.333933,-8.964766,-0.062599,4.675794e-13,0.0,numeric,-147.807381,-1.946423,-1.946423,0.000525,1.787886,0.15129,5.227856,0.027757,0.00135,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,21c63271de524214967f40336aed68d5,38163.345054
1,260f8a9a8994498982d42b7eed231a7b,1c7a2a6e7ae14c06b025ffb1659d73e4,13332,124,124,0.008065,11.135529,11.135529,0.008065,119.00813,1792.789004,6.402077,6.402077,0.322581,36.016924,-2.690683,2.491526,2.491526,0.322581,8.988515,-2.690683,2.491526,2.491526,0.322581,8.988515,0.004468,6.402689,6.402689,0.032258,36.021642,14.754817,5.331932,5.331932,0.322581,27.988896,0.3669,0.729624,0.729624,0.895161,-0.15619,2.936962,-0.735893,-0.735893,0.322581,-1.180441,0.02335,5.590404,5.590404,0.322581,30.057257,0.037184,5.239341,5.239341,0.951613,25.13529,411.53012,3.831147,3.831147,0.25,14.60629,38.765662,6.121918,6.121918,0.66129,42.356203,sum,0,numeric,-331.248575,-0.704291,-0.704291,0.000525,-1.504039,0.333933,-8.964766,-0.062599,4.675794e-13,0.0,numeric,-147.807381,-1.946423,-1.946423,0.000525,1.787886,0.15129,5.227856,0.027757,0.00135,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,227ad42c8e84480db5ce1fce8ca2f846,38183.198311
2,260f8a9a8994498982d42b7eed231a7b,1c7a2a6e7ae14c06b025ffb1659d73e4,13332,124,124,0.008065,11.135529,11.135529,0.008065,119.00813,1792.789004,6.402077,6.402077,0.322581,36.016924,-2.690683,2.491526,2.491526,0.322581,8.988515,-2.690683,2.491526,2.491526,0.322581,8.988515,0.004468,6.402689,6.402689,0.0322

In [349]:
set(model_df.columns)&set(merge_df.columns)

{'model_id'}

In [350]:
merge_df = merge_df.merge(model_df)
merge_df.head()


,dataset_id,transformation_id,rows,columns,feature_columns_size,target_mean,target_var,target_skew,target_perc_unique,target_kurtosis,mean_mean,mean_var,mean_skew,mean_perc_unique,mean_kurtosis,var_mean,var_var,var_skew,var_perc_unique,var_kurtosis,skew_mean,skew_var,skew_skew,skew_perc_unique,skew_kurtosis,perc_unique_mean,perc_unique_var,perc_unique_skew,perc_unique_perc_unique,perc_unique_kurtosis,kurtosis_mean,kurtosis_var,kurtosis_skew,kurtosis_perc_unique,kurtosis_kurtosis,perc_of_values_mode_mean,perc_of_values_mode_var,perc_of_values_mode_skew,perc_of_values_mode_perc_unique,perc_of_values_mode_kurtosis,target_slope_mean,target_slope_var,target_slope_skew,target_slope_perc_unique,target_slope_kurtosis,target_r_value_mean,target_r_value_var,target_r_value_skew,target_r_value_perc_unique,target_r_value_kurtosis,target_p_value_mean,target_p_value_var,target_p_value_skew,target_p_value_perc_unique,target_p_value_kurtosis,nunique_mean,nunique_var,nunique_skew,nunique_perc_unique,nunique_kurtosis,target_f_stat_mean,target_f_stat_var,target_f_stat_skew,target_f_stat_perc_unique,target_f_stat_kurtosis,transformation_type,input_column_0_target,input_column_0_type,input_column_0_mean,input_column_0_var,input_column_0_skew,input_column_0_perc_unique,input_column_0_kurtosis,input_column_0_perc_of_values_mode,input_column_0_target_slope,input_column_0_target_r_value,input_column_0_target_p_value,input_column_1_target,input_column_1_type,input_column_1_mean,input_column_1_var,input_column_1_skew,input_column_1_perc_unique,input_column_1_kurtosis,input_column_1_perc_of_values_mode,input_column_1_target_slope,input_column_1_target_r_value,input_column_1_target_p_value,input_column_0_nunique,input_column_0_target_f_stat,max_features,norm,analyzer,max_df,binary,use_idf,ngram_range,num_of_output_columns,model_id,validation_metrics,model_type,alpha,l1_ratio,objective,boosting_type,num_leaves,learning_rate,n_estimators
0,260f8a9a8994498982d42b7eed231a7b,1c7a2a6e7ae14c06b025ffb1659d73e4,13332,124,124,0.008065,11.135529,11.135529,0.008065,119.00813,1792.789004,6.402077,6.402077,0.322581,36.016924,-2.690683,2.491526,2.491526,0.322581,8.988515,-2.690683,2.491526,2.491526,0.322581,8.988515,0.004468,6.402689,6.402689,0.032258,36.021642,14.754817,5.331932,5.331932,0.322581,27.988896,0.3669,0.729624,0.729624,0.895161,-0.15619,2.936962,-0.735893,-0.735893,0.322581,-1.180441,0.02335,5.590404,5.590404,0.322581,30.057257,0.037184,5.239341,5.239341,0.951613,25.13529,411.53012,3.831147,3.831147,0.25,14.60629,38.765662,6.121918,6.121918,0.66129,42.356203,sum,0,numeric,-331.248575,-0.704291,-0.704291,0.000525,-1.504039,0.333933,-8.964766,-0.062599,4.675794e-13,0.0,numeric,-147.807381,-1.946423,-1.946423,0.000525,1.787886,0.15129,5.227856,0.027757,0.00135,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,21c63271de524214967f40336aed68d5,38163.345054,ElasticNet,1.607837,0.600348,NaN,NaN,NaN,NaN,NaN
1,260f8a9a8994498982d42b7eed231a7b,1c7a2a6e7ae14c06b025ffb1659d73e4,13332,124,124,0.008065,11.135529,11.135529,0.008065,119.00813,1792.789004,6.402077,6.402077,0.322581,36.016924,-2.690683,2.491526,2.491526,0.322581,8.988515,-2.690683,2.491526,2.491526,0.322581,8.988515,0.004468,6.402689,6.402689,0.032258,36.021642,14.754817,5.331932,5.331932,0.322581,27.988896,0.3669,0.729624,0.729624,0.895161,-0.15619,2.936962,-0.735893,-0.735893,0.322581,-1.180441,0.02335,5.590404,5.590404,0.322581,30.057257,0.037184,5.239341,5.239341,0.951613,25.13529,411.53012,3.831147,3.831147,0.25,14.60629,38.765662,6.121918,6.121918,0.66129,42.356203,sum,0,numeric,-331.248575,-0.704291,-0.704291,0.000525,-1.504039,0.333933,-8.964766,-0.062599,4.675794e-13,0.0,numeric,-147.807381,-1.946423,-1.946423,0.000525,1.787886,0.15129,5.227856,0.027757,0.00135,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,21c63271de524214967f40336aed68d5,38199.956146,ElasticNet,1.607837,0.600348,NaN,NaN,NaN,NaN,NaN
2,260f8a9a8994498982d42b7eed231a7b,1c7a2a6e7ae14c06b025ffb1659d73e4,13332,124,124,0.008065,11.135529,11.135529,0.00806

In [353]:
result_df[(result_df['dataset_id'] == '260f8a9a8994498982d42b7eed231a7b')&(result_df['model_id'] == '21c63271de524214967f40336aed68d5')]

,model_id,dataset_id,validation_metrics
0,21c63271de524214967f40336aed68d5,260f8a9a8994498982d42b7eed231a7b,38163.345054
10,21c63271de524214967f40336aed68d5,260f8a9a8994498982d42b7eed231a7b,38199.956146
20,21c63271de524214967f40336aed68d5,260f8a9a8994498982d42b7eed231a7b,38082.021069
30,21c63271de524214967f40336aed68d5,260f8a9a8994498982d42b7eed231a7b,37589.436341
40,21c63271de524214967f40336aed68d5,260f8a9a8994498982d42b7eed231a7b,38197.062348
50,21c63271de524214967f40336aed68d5,260f8a9a8994498982d42b7eed231a7b,37431.546144
60,21c63271de524214967f40336aed68d5,260f8a9a8994498982d42b7eed231a7b,37581.787445
70,21c63271de524214967f40336aed68d5,260f8a9a8994498982d42b7eed231a7b,37893.186062
80,21c63271de524214967f40336aed68d5,260f8a9a8994498982d42b7eed231a7b,38193.276718
90,21c63271de524214967f40336aed68d5,260f8a9a8994498982d42b7eed231a7b,37785.907866


In [354]:
result_df.shape

(1375, 3)

In [352]:
merge_df[['dataset_id', 'transformation_id', 'model_id', 'validation_metrics']].head(20)

,dataset_id,transformation_id,model_id,validation_metrics
0,260f8a9a8994498982d42b7eed231a7b,1c7a2a6e7ae14c06b025ffb1659d73e4,21c63271de524214967f40336aed68d5,38163.345054
1,260f8a9a8994498982d42b7eed231a7b,1c7a2a6e7ae14c06b025ffb1659d73e4,21c63271de524214967f40336aed68d5,38199.956146
2,260f8a9a8994498982d42b7eed231a7b,1c7a2a6e7ae14c06b025ffb1659d73e4,21c63271de524214967f40336aed68d5,38082.021069
3,260f8a9a8994498982d42b7eed231a7b,1c7a2a6e7ae14c06b025ffb1659d73e4,21c63271de524214967f40336aed68d5,37589.436341
4,260f8a9a8994498982d42b7eed231a7b,1c7a2a6e7ae14c06b025ffb1659d73e4,21c63271de524214967f40336aed68d5,38197.062348
5,260f8a9a8994498982d42b7eed231a7b,1c7a2a6e7ae14c06b025ffb1659d73e4,21c63271de524214967f40336aed68d5,37431.546144
6,260f8a9a8994498982d42b7eed231a7b,1c7a2a6e7ae14c06b025ffb1659d73e4,21c63271de524214967f40336aed68d5,37581.787445
7,260f8a9a8994498982d42b7eed231a7b,1c7a2a6e7ae14c06b025ffb1659d73e4,21c63271de524214967f40336aed68d5,37893.186062
8,260f8a9a8994498982d42b7eed231a7b,1c7a2a6e7ae14c06b025ffb1659d73e4,21c63271de524214967f40336aed68d5,38193.276718
9,260f8a9a8994498982d42b7eed231a7b,1c7a2a6e7ae14c06b025ffb1659d73e4,21c63271de524214967f40336aed68d5,37785.907866


In [326]:
merge_df = merge_df.drop(['model_id', 'dataset_id', 'transformation_id'], axis = 1)

In [327]:
merge_df.head()

,validation_metrics,transformation_type,input_column_0_target,input_column_0_type,input_column_0_mean,input_column_0_var,input_column_0_skew,input_column_0_perc_unique,input_column_0_kurtosis,input_column_0_perc_of_values_mode,input_column_0_target_slope,input_column_0_target_r_value,input_column_0_target_p_value,input_column_1_target,input_column_1_type,input_column_1_mean,input_column_1_var,input_column_1_skew,input_column_1_perc_unique,input_column_1_kurtosis,input_column_1_perc_of_values_mode,input_column_1_target_slope,input_column_1_target_r_value,input_column_1_target_p_value,num_of_output_columns,input_column_0_nunique,input_column_0_target_f_stat,max_features,norm,analyzer,max_df,binary,use_idf,ngram_range,rows,columns,feature_columns_size,target_mean,target_var,target_skew,target_perc_unique,target_kurtosis,mean_mean,mean_var,mean_skew,mean_perc_unique,mean_kurtosis,var_mean,var_var,var_skew,var_perc_unique,var_kurtosis,skew_mean,skew_var,skew_skew,skew_perc_unique,skew_kurtosis,perc_unique_mean,perc_unique_var,perc_unique_skew,perc_unique_perc_unique,perc_unique_kurtosis,kurtosis_mean,kurtosis_var,kurtosis_skew,kurtosis_perc_unique,kurtosis_kurtosis,perc_of_values_mode_mean,perc_of_values_mode_var,perc_of_values_mode_skew,perc_of_values_mode_perc_unique,perc_of_values_mode_kurtosis,target_slope_mean,target_slope_var,target_slope_skew,target_slope_perc_unique,target_slope_kurtosis,target_r_value_mean,target_r_value_var,target_r_value_skew,target_r_value_perc_unique,target_r_value_kurtosis,target_p_value_mean,target_p_value_var,target_p_value_skew,target_p_value_perc_unique,target_p_value_kurtosis,nunique_mean,nunique_var,nunique_skew,nunique_perc_unique,nunique_kurtosis,target_f_stat_mean,target_f_stat_var,target_f_stat_skew,target_f_stat_perc_unique,target_f_stat_kurtosis,model_type,alpha,l1_ratio,objective,boosting_type,num_leaves,learning_rate,n_estimators
0,38089.386057,standard_scaler,0,numeric,-147.807381,-1.946423,-1.946423,0.000525,1.787886,0.15129,5.227856,0.027757,0.00135,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13332,124,124,0.008065,11.135529,11.135529,0.008065,119.00813,1792.789004,6.402077,6.402077,0.322581,36.016924,-2.690683,2.491526,2.491526,0.322581,8.988515,-2.690683,2.491526,2.491526,0.322581,8.988515,0.004468,6.402689,6.402689,0.032258,36.021642,14.754817,5.331932,5.331932,0.322581,27.988896,0.3669,0.729624,0.729624,0.895161,-0.15619,2.936962,-0.735893,-0.735893,0.322581,-1.180441,0.02335,5.590404,5.590404,0.322581,30.057257,0.037184,5.239341,5.239341,0.951613,25.13529,411.53012,3.831147,3.831147,0.25,14.60629,38.765662,6.121918,6.121918,0.66129,42.356203,ElasticNet,0.537285,0.23927,NaN,NaN,NaN,NaN,NaN
1,38272.772091,standard_scaler,0,numeric,-147.807381,-1.946423,-1.946423,0.000525,1.787886,0.15129,5.227856,0.027757,0.00135,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13332,124,124,0.008065,11.135529,11.135529,0.008065,119.00813,1792.789004,6.402077,6.402077,0.322581,36.016924,-2.690683,2.491526,2.491526,0.322581,8.988515,-2.690683,2.491526,2.491526,0.322581,8.988515,0.004468,6.402689,6.402689,0.032258,36.021642,14.754817,5.331932,5.331932,0.322581,27.988896,0.3669,0.729624,0.729624,0.895161,-0.15619,2.936962,-0.735893,-0.735893,0.322581,-1.180441,0.02335,5.590404,5.590404,0.322581,30.057257,0.037184,5.239341,5.239341,0.951613,25.13529,411.53012,3.831147,3.831147,0.25,14.60629,38.765662,6.121918,6.121918,0.66129,42.356203,ElasticNet,0.537285,0.23927,NaN,NaN,NaN,NaN,NaN
2,38178.645415,standard_scaler,0,numeric,-147.807381,-1.946423,-1.946423,0.000525,1.787886,0.15129,5.227856,0.027757,0.00135,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13332,124,124,0.008065,11.135529,11.135529,0.008065,119.00813,1792.789004,6.402077,6.402077,0.322581,36.016924,-2.690683,2.491526,2.491526,0.322581,8.988515,-2.690683,2.491526,2.491526,0.322581,8.988515,0.004468,6.402689,6.402689,0.032258,36.021

In [328]:
x = merge_df.drop('validation_metrics', axis = 1)
y = merge_df['validation_metrics']


In [307]:
merge_df.to_csv(r'C:\Users\TristanDelforge\Documents\data_files\metadata1.csv', index = False)


In [308]:
corr = merge_df.corr()
corr2 = corr.applymap(lambda x: x*x)
corr2

,validation_metrics,input_column_0_target,input_column_0_mean,input_column_0_var,input_column_0_skew,input_column_0_perc_unique,input_column_0_kurtosis,input_column_0_perc_of_values_mode,input_column_0_target_slope,input_column_0_target_r_value,input_column_0_target_p_value,input_column_1_target,input_column_1_mean,input_column_1_var,input_column_1_skew,input_column_1_perc_unique,input_column_1_kurtosis,input_column_1_perc_of_values_mode,input_column_1_target_slope,input_column_1_target_r_value,input_column_1_target_p_value,num_of_output_columns,input_column_0_nunique,input_column_0_target_f_stat,max_features,max_df,rows,columns,feature_columns_size,target_mean,target_var,target_skew,target_perc_unique,target_kurtosis,mean_mean,mean_var,mean_skew,mean_perc_unique,mean_kurtosis,var_mean,var_var,var_skew,var_perc_unique,var_kurtosis,skew_mean,skew_var,skew_skew,skew_perc_unique,skew_kurtosis,perc_unique_mean,perc_unique_var,perc_unique_skew,perc_unique_perc_unique,perc_unique_kurtosis,kurtosis_mean,kurtosis_var,kurtosis_skew,kurtosis_perc_unique,kurtosis_kurtosis,perc_of_values_mode_mean,perc_of_values_mode_var,perc_of_values_mode_skew,perc_of_values_mode_perc_unique,perc_of_values_mode_kurtosis,target_slope_mean,target_slope_var,target_slope_skew,target_slope_perc_unique,target_slope_kurtosis,target_r_value_mean,target_r_value_var,target_r_value_skew,target_r_value_perc_unique,target_r_value_kurtosis,target_p_value_mean,target_p_value_var,target_p_value_skew,target_p_value_perc_unique,target_p_value_kurtosis,nunique_mean,nunique_var,nunique_skew,nunique_perc_unique,nunique_kurtosis,target_f_stat_mean,target_f_stat_var,target_f_stat_skew,target_f_stat_perc_unique,target_f_stat_kurtosis,alpha,l1_ratio,num_leaves,learning_rate,n_estimators
validation_metrics,NaN,NaN,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,NaN,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,NaN,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,9.801443e-03,4.852457e-03,0.000000e+00,0.000000e+00,0.000000e+00
input_column_0_target,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
input_column_0_mean,0.000000,NaN,1.000000e+00,7.587989e-03,7.587989e-03,9.507109e-03,2.884023e-04,6.451320e-03,3.281560e-05,7.709157e-03,5.908910e-04,NaN,5.275695e-05,1.168524e-05,1.168524e-05,3.176914e-05,6.858006e-05,9.592378e-05,6.971294e-05,1.146993e-03,4.215869e-04,2.252630e-02,NaN,NaN,NaN,NaN,NaN,6.992619e-05,6.992619e-05,5.322993e-04,1.495217e-04,1.495217e-04,5.322993e-04,6.991868e-05,1.084727e-05,1.942997e-03,1.942997e-03,3.988328e-04,1.693356e-03,2.431718e-05,2.275094e-03,2.275094e-03,5.335454e-04,1.366404e-03,2.431718e-05,2.275094e-03,2.275094e-03,5.335454e-04,1.366404e-03,7.665178e-04,4.953207e-04,4.953207e-04,1.

In [309]:
corr2.sort_values('validation_metrics', ascending = False)[['validation_metrics']]

,validation_metrics
alpha,0.009801
l1_ratio,0.004852
input_column_0_mean,0.000000
perc_of_values_mode_var,0.000000
target_slope_kurtosis,0.000000
target_slope_perc_unique,0.000000
target_slope_skew,0.000000
target_slope_var,0.000000
target_slope_mean,0.000000
perc_of_values_mode_kurtosis,0.000000
